In [1]:
import matplotlib.pyplot as plt

from data.LSA_Dataset import LSA_Dataset
from data.transforms import get_frames_reduction_transform, get_roi_selector_transform, keypoint_format_transform

In [5]:
root = '/mnt/data/datasets/cn_sordos_db/data/cuts'
max_frames = 75

dataset = LSA_Dataset(
    root,
    load_videos = False,
    frame_transform = get_roi_selector_transform(128,128),
    video_transform = get_frames_reduction_transform(max_frames),
    keypoints_transform = get_frames_reduction_transform(max_frames),
    keypoints_transform_each = keypoint_format_transform
    )

In [6]:
import random

i = random.randint(0, dataset.__len__())
sample = dataset.__getitem__(i)

In [7]:
print(sample[3])
clip = sample[1]
print("[F, C, H, W]", clip.size())
plt.figure(figsize=(12, 12))
for i in range(20):
    plt.subplot(5, 5, i + 1)
    iclip = clip[i, ...].permute(1,2,0)
    plt.imshow(iclip)
    plt.axis("off")

AttributeError: 'list' object has no attribute 'tokens'

In [10]:
keys = list(sample[2])

In [11]:
from torch import stack

print(sample[3])

print(len(keys))
print(keys[0].size())

vid_keys = stack(keys)
print(vid_keys.size())


['la', 'otra', 'asociación', 'fue', 'la', 'de', 'quilmes', ',', '33', 'años', '.']
75
torch.Size([3, 42])
torch.Size([75, 3, 42])


In [19]:
from typing import List
from torch import Tensor, nn, stack
from torch.nn.functional import relu
import torch

from model.PositionalEncoding import PositionalEncoding


class KeypointModel(nn.Module):

    def __init__(self, inp_max_len: int, tgt_max_len: int):
        super(KeypointModel, self).__init__()

        # in_features is the result of flattening the input of (x,y,c).(k1, ..., k42)
        self.fc = nn.Linear(in_features=126, out_features=64)
        kernel_size = 5
        self.conv1d = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=kernel_size)
        self.pe = PositionalEncoding(d_model=32, max_len=(inp_max_len - kernel_size + 1))
        self.transformer = nn.Transformer()
        

    def forward(self, inp: List[Tensor], tgt: Tensor):
        # flatten and apply fc frame by frame, then stack the frames and permute dims for conv
        x = stack([relu(self.fc(torch.flatten(frame))) for frame in inp]).permute(1,0)
        # unsqueeze adds dimention representing the batch CHEQUEAR
        x = self.conv1d(x.unsqueeze(0))[0].permute(1,0)
        # unsqueeze adds dimention representing the batch CHEQUEAR
        x = x.unsqueeze(1)
        x = self.pe(x)
        #x = self.transformer(x)

        return x

In [21]:

model = KeypointModel(75,5)
print(model(vid_keys, None).size())

torch.Size([71, 1, 32])


In [ ]:
print(torch.arange(5).size())
print(torch.arange(5).unsqueeze(0).size())

torch.Size([5])
torch.Size([1, 5])


In [ ]:
loader = DataLoader(dataset, batch_size=12)
data: Dict[str, Any] = {'path': [], 'clip': [], 'keypoints': [], 'label': []}
for batch in loader:
    for i in range(len(batch['path'])):
        data['path'].append(batch['path'][i])
        data['clip'].append(batch['clip'][i].size())
        #data['keypoints'].append(batch['end'][i])
        data['label'].append(batch['label'][i])
print(data)

NameError: name 'DataLoader' is not defined